In [1]:
! pip install transformers trl py7zr optimum accelerate bitsandbytes
! pip install git+https://github.com/huggingface/peft
! pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118
!pip install transformers accelerate bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 10.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 24.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.3 MB/s eta 0:00:00:00:01

In [2]:
from transformers import (
BitsAndBytesConfig,
AutoModelForCausalLM,
AutoTokenizer,
TrainingArguments,
HfArgumentParser,
logging,
pipeline
)
import torch
from trl import SFTTrainer
from peft import PeftModel, LoraConfig
import pandas as pd
import numpy as np
import os
import re
from datasets import Dataset, DatasetDict

2025-07-13 13:56:51.688262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752415011.944984      78 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752415012.066201      78 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_excel("/kaggle/input/persiadata/Book22.xlsx")
df.head()

,question,answer
0,شخصی وضو گرفته ولی شک دارد که حدث از او سر زده...,می‌تواند نماز را با همان طهارت بخواند.
1,کسی که وضو داشته و حدثی هم یقینا از او سر زده ...,باید دوباره طهارت بگیرد.
2,آیا داشتن وضو برای خواندن همه نمازها واجب است؟,نماز باید با وضو یا با غسل مجزی از وضو (مانند ...
3,کسی که از روی جهل وضوی خود را به شکل ناصحیح ان...,چون وضو مقدمه نماز است، باید تمام نمازها را قض...
4,بعد از آمدن از مهمانی، فهمیدیم آب آن جا غصبی ب...,وضو با آب غصبی باطل است هرچند مکلف از غصبی بود...


In [4]:
df.shape

(50, 2)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  50 non-null     object
 1   answer    50 non-null     object
dtypes: object(2)
memory usage: 932.0+ bytes


In [6]:
df.isnull().sum()

question    0
answer      0
dtype: int64

In [7]:
df.duplicated().sum()

0

In [8]:
def text_cleaning(text):
  text = re.sub('[\u200c]', ' ', text)
  text = re.sub('[\n]', ' ', text)
  text = re.sub('[a-zA-Z]', ' ', text)
  text = re.sub('\s+', ' ', text)
  text = re.sub('[.]', '', text)
  return text.strip()

In [9]:
df['question'] = df['question'].apply(text_cleaning)
df['answer'] = df['answer'].apply(text_cleaning)

In [10]:
b = []
for i in range(df.shape[0]):
    question = df['question'][i]
    answer = df['answer'][i]
    text = f"### Human:{question}\n### Assistant: {answer}"
    b.append(text)

In [11]:
b

['### Human:شخصی وضو گرفته ولی شک دارد که حدث از او سر زده باشد، برای نماز چه وظیفه ای دارد؟\n### Assistant: می تواند نماز را با همان طهارت بخواند',
 '### Human:کسی که وضو داشته و حدثی هم یقینا از او سر زده ولی نمی داند کدام یک زودتر بوده، چه وظیفه ای دارد؟\n### Assistant: باید دوباره طهارت بگیرد',
 '### Human:آیا داشتن وضو برای خواندن همه نمازها واجب است؟\n### Assistant: نماز باید با وضو یا با غسل مجزی از وضو (مانند غسل جنابت) خوانده شود و بدون یکی از آن دو باطل است تنها نمازی که نیاز به وضو ندارد نماز میت می باشد',
 '### Human:کسی که از روی جهل وضوی خود را به شکل ناصحیح انجام داده است، نمازهایش چه حکمی دارد؟\n### Assistant: چون وضو مقدمه نماز است، باید تمام نمازها را قضا نماید',
 '### Human:بعد از آمدن از مهمانی، فهمیدیم آب آن جا غصبی بوده، آیا نمازی که با آن وضو خوانده ام قبول است؟\n### Assistant: وضو با آب غصبی باطل است هرچند مکلف از غصبی بودن آن بی اطلاع باشد وضو مقدمه نماز است و هرگاه وضو باطل باشد، نماز نیز باطل خواهد بود',
 '### Human:آیا در وضو گفتن قربة الی الله واجب است؟\n##

In [12]:
data = pd.DataFrame({
    'text': b
})

In [13]:
data.head()

,text
0,### Human:شخصی وضو گرفته ولی شک دارد که حدث از...
1,### Human:کسی که وضو داشته و حدثی هم یقینا از ...
2,### Human:آیا داشتن وضو برای خواندن همه نمازها...
3,### Human:کسی که از روی جهل وضوی خود را به شکل...
4,### Human:بعد از آمدن از مهمانی، فهمیدیم آب آن...


In [14]:
train_dict = DatasetDict({'train': Dataset.from_pandas(data)})

In [15]:
train_dict

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 50
    })
})

In [16]:
train_data = train_dict['train']

In [17]:
train_data

Dataset({
    features: ['text'],
    num_rows: 50
})

In [18]:
train_data[0]

{'text': '### Human:شخصی وضو گرفته ولی شک دارد که حدث از او سر زده باشد، برای نماز چه وظیفه ای دارد؟\n### Assistant: می تواند نماز را با همان طهارت بخواند'}

In [19]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [20]:
logging.set_verbosity_warning()

In [21]:
model_name = 'MehdiHosseiniMoghadam/AVA-Llama-3-V2'

In [22]:
new_model = 'MehdiHosseiniMoghadam/AVA-Llama-3-V2-test'

In [23]:
output_dir = '/kaggle/working/'

In [45]:
num_train_epochs = 20

In [25]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = False
)

In [26]:
torch.cuda.empty_cache()

In [27]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = {"": 0}
)

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [28]:
model.config.use_cache = False

In [29]:
model.config.pretraining_tp = 1

In [30]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True, use_fast = False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

In [31]:
def text_generation(pr):
    pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 100)
    prompt = f"### Human:{pr}\n### Assistant:"
    result = pipe(prompt)
    print(result[0]['generated_text'])

In [32]:
text_generation('شخصی وضو گرفته ولی شک دارد که حدث از او سر زده باشد، برای نماز چه وظیفه ای دارد؟')

Device set to use cuda:0


### Human:شخصی وضو گرفته ولی شک دارد که حدث از او سر زده باشد، برای نماز چه وظیفه ای دارد؟
### Assistant:در این مورد، باید از شبهات و شک در مورد حدث از او سر زدن شخص، فاصله بگیرید و به عنوان یک مسلمان متعهد عمل کنید. از آنجا که شخص وضو گرفته است، وظیفه او نماز خواندن است و باید نماز خود را به طور معمول انجام دهد. به عنوان


In [33]:
text_generation('کسی که وضو داشته و حدثی هم یقینا از او سر زده ولی نمی داند کدام یک زودتر بوده، چه وظیفه ای دارد؟')

Device set to use cuda:0


### Human:کسی که وضو داشته و حدثی هم یقینا از او سر زده ولی نمی داند کدام یک زودتر بوده، چه وظیفه ای دارد؟
### Assistant:


In [34]:
text_generation('آیا داشتن وضو برای خواندن همه نمازها واجب است؟')

Device set to use cuda:0


### Human:آیا داشتن وضو برای خواندن همه نمازها واجب است؟
### Assistant:سلام


In [35]:
text_generation('کسی که از روی جهل وضوی خود را به شکل ناصحیح انجام داده است، نمازهایش چه حکمی دارد؟')

Device set to use cuda:0


### Human:کسی که از روی جهل وضوی خود را به شکل ناصحیح انجام داده است، نمازهایش چه حکمی دارد؟
### Assistant:در مواردی که شخصی از روی جهل و ضعف خود را به شکل ناصحیح انجام می دهد، نمازهای او باطل نیست. اسلام به عنوان یک دین متقبل از خطاهاست و کسانی که به اشتباه یا جهل نماز میخوانند، بازگشت به اسلام


In [36]:
text_generation('بعد از آمدن از مهمانی، فهمیدیم آب آن جا غصبی بوده، آیا نمازی که با آن وضو خوانده ام قبول است؟')

Device set to use cuda:0


### Human:بعد از آمدن از مهمانی، فهمیدیم آب آن جا غصبی بوده، آیا نمازی که با آن وضو خوانده ام قبول است؟
### Assistant:سلام
اگر آب غصبی بود، نماز وضو خوانده ای را قبول نیست. در اسلام، برای نماز وضو خواندن نیاز به استفاده از آب پاک و غصبی نیست. در صورت استفاده از آب غصبی، باید دوباره وضو بگیرید. برای نماز و


In [37]:
torch.cuda.empty_cache()

In [38]:
def preprocess_function(examples):
    return tokenizer(
        examples['text'],
        truncation = True,
        padding = 'max_length',
        max_length = 256
    )

In [39]:
tokenized_dataset = train_data.map(preprocess_function, batched = True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [40]:
peft_config = LoraConfig(
    lora_alpha = 16,
    r = 64,
    bias = 'none',
    task_type = 'CAUSAL_LM'
)

In [46]:
training_arguments = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,
    optim = 'paged_adamw_8bit',
    save_steps = 0,
    logging_steps = 1,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    fp16 = True,
    bf16 = False,
    max_grad_norm = 0.3,
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    lr_scheduler_type = 'cosine',
    report_to = 'tensorboard'
)

In [47]:
trainer = SFTTrainer(
    model = model,
    train_dataset = tokenized_dataset,
    peft_config = peft_config,
    args = training_arguments
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [48]:
torch.cuda.empty_cache()

In [49]:
print('Starting training...')
trainer.train()
print('Training complete...')

Starting training...


Step,Training Loss
1,5.034900
2,4.561700
3,4.771200
4,5.099000
5,4.996700
6,4.732200
7,3.742300
8,3.333900
9,2.038300
10,0.927000


Training complete...


In [50]:
trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('MehdiHosseiniMoghadam/AVA-Llama-3-V2-test/tokenizer_config.json',
 'MehdiHosseiniMoghadam/AVA-Llama-3-V2-test/special_tokens_map.json',
 'MehdiHosseiniMoghadam/AVA-Llama-3-V2-test/chat_template.jinja',
 'MehdiHosseiniMoghadam/AVA-Llama-3-V2-test/tokenizer.json')

In [52]:
def text_generation2(pr):
    pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 100)
    prompt = f"### Human:{pr}\n### Assistant:"
    result = pipe(prompt)
    print(result[0]['generated_text'])

In [54]:
text_generation2('شخصی وضو گرفته ولی شک دارد که حدث از او سر زده باشد، برای نماز چه وظیفه ای دارد؟')

Device set to use cuda:0


### Human:شخصی وضو گرفته ولی شک دارد که حدث از او سر زده باشد، برای نماز چه وظیفه ای دارد؟
### Assistant: می تواند نماز را با همان طهارت بخواند


In [57]:
text_generation2('کسی که وضو داشته و حدثی هم یقینا از او سر زده ولی نمی داند کدام یک زودتر بوده، چه وظیفه ای دارد؟')

Device set to use cuda:0


### Human:کسی که وضو داشته و حدثی هم یقینا از او سر زده ولی نمی داند کدام یک زودتر بوده، چه وظیفه ای دارد؟
### Assistant: باید دوباره طهارت بگیرد


In [59]:
text_generation2('آیا داشتن وضو برای خواندن همه نمازها واجب است؟')

Device set to use cuda:0


### Human:آیا داشتن وضو برای خواندن همه نمازها واجب است؟
### Assistant: نماز باید با وضو یا با غسل مجزی از وضو (مانند غسل جنابت) خوانده شود و بدون یکی از آن دو باطل است تنها نمازی که نیاز به وضو ندارد نماز میت می باشد


In [60]:
text_generation2('کسی که از روی جهل وضوی خود را به شکل ناصحیح انجام داده است، نمازهایش چه حکمی دارد؟')

Device set to use cuda:0


### Human:کسی که از روی جهل وضوی خود را به شکل ناصحیح انجام داده است، نمازهایش چه حکمی دارد؟
### Assistant: چون وضو مقدمه نماز است، باید تمام نمازها را قضا نماید


In [61]:
text_generation2('بعد از آمدن از مهمانی، فهمیدیم آب آن جا غصبی بوده، آیا نمازی که با آن وضو خوانده ام قبول است؟')

Device set to use cuda:0


### Human:بعد از آمدن از مهمانی، فهمیدیم آب آن جا غصبی بوده، آیا نمازی که با آن وضو خوانده ام قبول است؟
### Assistant: وضو با آب غصبی باطل است هرچند مکلف از غصبی بودن آن بی اطلاع باشد وضو باید تجدید شود


In [65]:
text_generation2('آیا نیت برای وضو واجب است و اگر نماز با وضوی بدون نیت خوانده شود آیا اشكال دارد؟')

Device set to use cuda:0


### Human:آیا نیت برای وضو واجب است و اگر نماز با وضوی بدون نیت خوانده شود آیا اشكال دارد؟
### Assistant: وضو واجب است و نیت برای انجام وضو واجب نمازی نیز واجب است


In [72]:
def text_generation3(pr):
    pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 100)
    prompt = f"### Human:{pr}\n### Assistant:"
    result = pipe(prompt,
                 do_sample = True,
                 top_k = 10,
                 temperature = 0.7,
                 top_p = 0.95,
                 eos_token_id = tokenizer.eos_token_id,
                 num_return_sequences = 1)
    print(result[0]['generated_text'])

In [68]:
text_generation3('شخصی وضو گرفته ولی شک دارد که حدث از او سر زده باشد، برای نماز چه وظیفه ای دارد؟')

Device set to use cuda:0


### Human:شخصی وضو گرفته ولی شک دارد که حدث از او سر زده باشد، برای نماز چه وظیفه ای دارد؟
### Assistant: می تواند نماز را با همان وضو خواند


In [74]:
text_generation3('کسی که وضو داشته و حدثی هم یقینا از او سر زده ولی نمی داند کدام یک زودتر بوده، چه وظیفه ای دارد؟')

Device set to use cuda:0


### Human:کسی که وضو داشته و حدثی هم یقینا از او سر زده ولی نمی داند کدام یک زودتر بوده، چه وظیفه ای دارد؟
### Assistant: باید دوباره طهارت بگیرد


In [75]:
text_generation3('آیا داشتن وضو برای خواندن همه نمازها واجب است؟')

Device set to use cuda:0


### Human:آیا داشتن وضو برای خواندن همه نمازها واجب است؟
### Assistant: نماز باید با وضو یا با غسل مجزی از وضو (مانند غسل جنابت) خوانده شود و بدون یکی از آن دو باطل است تنها نمازی که نیاز به وضو ندارد نماز میت می باشد


In [77]:
text_generation3('کسی که از روی جهل وضوی خود را به شکل ناصحیح انجام داده است، نمازهایش چه حکمی دارد؟')

Device set to use cuda:0


### Human:کسی که از روی جهل وضوی خود را به شکل ناصحیح انجام داده است، نمازهایش چه حکمی دارد؟
### Assistant: چون وضو مقدمه نماز است، باید تمام نمازها را قضا کند


In [78]:
text_generation3('نماز کسی که وضوی ناصحیح بگیرد چه حکمی دارد؟')

Device set to use cuda:0


### Human:نماز کسی که وضوی ناصحیح بگیرد چه حکمی دارد؟
### Assistant: نمازی که با وضو ناصحیح انجام شود، باطل می باشد و نمازگزار باید وضو را تجدید کند


In [79]:
model2 = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = {"": 0}
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [80]:
def text_generation_un(pr):
    pipe = pipeline(task = 'text-generation', model = model2, tokenizer = tokenizer, max_length = 100)
    prompt = f"### Human:{pr}\n### Assistant:"
    result = pipe(prompt)
    print(result[0]['generated_text'])

In [94]:
text_generation_un('آیا جوهر خودکار مانع وضو به حساب می آید؟')

Device set to use cuda:0


### Human:آیا جوهر خودکار مانع وضو به حساب می آید؟
### Assistant:فارسی
من به عنوان یک مدل زبان هوش مصنوعی، پاسخ به سؤال های شما را فراهم می کنم. اگر شما سؤال دیگری دارید، لطفا سؤال کنید.
با این وجود، باید به این نکته توجه داشته باشید که مدل های هوش مصنوعی مانند من، فقط برای اطلاعات و راهنمایی های کلی طراحی شده اند و نمی توانند


In [93]:
text_generation3('آیا جوهر خودکار مانع وضو به حساب می آید؟')

Device set to use cuda:0


### Human:آیا جوهر خودکار مانع وضو به حساب می آید؟
### Assistant: اگر جوهر خودکار از نوع جوهرهای مس می باشد، مانع وضو و طهارت می شود


In [95]:
text_generation_un('در مسح پا تر شدن ناخن باید به چه شکلی باشد؟')

Device set to use cuda:0


### Human:در مسح پا تر شدن ناخن باید به چه شکلی باشد؟
### Assistant:در این مورد، معمولاً توصیه می شود که ناخن ها به شکل طبیعی و طبیعی باشند یا به شکل طبیعی اما کوتاه تر باشند. کوتاه کردن ناخن های پا می تواند به کاهش خطر بیماری ها کمک کند و همچنین ظاهرمحترمتری به ناخن ها می دهد. در صورتی که ناخن های پا به شکل طبیعی بلند هستند،


In [100]:
text_generation3('در مسح پا تر شدن ناخن باید به چه شکلی باشد؟')

Device set to use cuda:0


### Human:در مسح پا تر شدن ناخن باید به چه شکلی باشد؟
### Assistant: ناخن ها نباید از حد متعارف بیش تر تر شوند در این صورت می توانیم پا را مسح کنیم


In [ ]:
کسی که بدون نیت، وضو گرفته است، آیا باطل است؟

In [115]:
text_generation3('کسی که بدون نیت، وضو گرفته است، آیا باطل است؟')

Device set to use cuda:0


### Human:کسی که بدون نیت، وضو گرفته است، آیا باطل است؟
### Assistant: بله باطل است زیرا احکام وضو به نیت و قصد انسان بستگی دارد


In [108]:
text_generation_un('کسی که بدون نیت، وضو گرفته است، آیا باطل است؟')

Device set to use cuda:0


### Human:کسی که بدون نیت، وضو گرفته است، آیا باطل است؟
### Assistant:در اسلام، شایستگی وضو در میان مسلمانان بسیار مهم است. شایستگی وضو مهمترین شرط برای نماز است و بدون آن، نماز نیز باطل است. در مورد کسی که بدون نیت، وضو گرفته است، فتوی معمولاً این است که این وضو باطل است.

در اسلام، نیت مهمترین شرط برای انجام هر
